# Exploration des données

## Importation des librairies

In [1]:
import pandas as pd
import pickle
import gzip
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split

## Chargement des données

In [2]:
train_path = "../data/train.csv"
test_path = "../data/test.csv"
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

## Analyse des données
### Taille des données

In [3]:
print(
    f'taille du jeu train :\n'
    f'nb de lignes :\t\t{train_df.shape[0]}\n'
    f'nb de colonnes :\t{train_df.shape[1]}\n\n'
    f'taille du jeu test :\n'
    f'nb de lignes :\t\t{test_df.shape[0]}\n'
    f'nb de colonnes :\t{test_df.shape[1]}'
)

taille du jeu train :
nb de lignes :		1458110
nb de colonnes :	5

taille du jeu test :
nb de lignes :		393968
nb de colonnes :	4


### Colonnes

In [4]:
train_df.columns

Index(['Unnamed: 0', 'id', 'global_rate', 'establishment_id', 'review_text'], dtype='object')

In [5]:
train_df.dtypes

Unnamed: 0            int64
id                    int64
global_rate         float64
establishment_id      int64
review_text          object
dtype: object

In [6]:
test_df.columns

Index(['Unnamed: 0', 'id', 'establishment_id', 'review_text'], dtype='object')

In [7]:
test_df.dtypes

Unnamed: 0           int64
id                   int64
establishment_id     int64
review_text         object
dtype: object

### Aperçu

In [8]:
train_df.sample(20)

,Unnamed: 0,id,global_rate,establishment_id,review_text
530569,690381,13512007,8.0,15759,Ma collègue et moi y avons dormi une nuit. Nou...
1004039,1267569,27491545,10.0,24185,Une bonne surprise que la découverte de ce pet...
125354,161302,2370574,8.0,10339,"Bonne soirée, rien à redire sur le menu à 29 €..."
1431346,1820255,72256173,4.0,38288,"Cadre extérieur très sympa, au calme. Parking ..."
477630,625592,16472132,8.0,13817,Un hôtel bien situé. L’accueil est assuré à to...
392252,516062,10379879,8.0,12009,Nous avons passer un séminaire sur place avec ...
121478,150235,34746049,6.0,10332,"Belle vue, endroit sympathique bien décoré. No..."
1280260,1617843,69479813,8.0,26356,NaN
397245,523634,67373478,2.0,12084,O/5
1053180,1331697,77894954,10.0,24398,Nous avons apprécié chaque moment durant ce re...


In [9]:
test_df.sample(20)

,Unnamed: 0,id,establishment_id,review_text
361108,1723422,69455501,29107,NaN
179780,840240,69581301,20678,NaN
246181,1161716,78503348,23331,Thalassa oleron\nUn gag à fuir\nComment dire ...
178955,839415,34882249,20678,Pas d'eau chaude 1er et 2ème jour. Verres sale...
370658,1750197,74791376,30711,NaN
187824,853627,77940697,20739,NaN
172361,764206,15723399,17132,On y va toujours pour la cuisine de grande qua...
378095,1770149,30957128,32037,NaN
275749,1320355,74107218,24348,"Accueil chaleureux, des plats qui régalent le..."
61571,264286,71288719,11241,Très bel hôtel placé magnifiquement bien en pl...


Il faudra supprimer les lignes avec des valeurs manquantes pour l'ensemble de train et de test.

### Analyse de la feature d'intérêt (texte)

Vérification de l'homogénéité de la feature texte

In [10]:
def is_not_string(value):
    return not isinstance(value, str)

In [11]:
train_df[train_df['review_text'].apply(is_not_string)]['review_text'].unique()

array([nan], dtype=object)

In [12]:
test_df[test_df['review_text'].apply(is_not_string)]['review_text'].unique()

array([nan], dtype=object)

Seules les valeurs non manquantes de la feature d'intérêt ne sont pas des strings.

### Répartition des données par établissements (en vue de la sélection ou non comme feature du modèle)

In [13]:
train_df['establishment_id'].value_counts()

establishment_id
26354    14205
23346    10697
21671     8798
25324     8426
14107     8330
         ...  
17042        1
16913        1
23183        1
23185        1
23189        1
Name: count, Length: 2368, dtype: int64

In [14]:
fig = px.histogram(
    x=train_df['establishment_id'].value_counts().index,
    y=train_df['establishment_id'].value_counts().values,
    nbins=1000
)
fig.update_layout(
    title="Number of establishment per id",
    xaxis_title="Establishment id",
    yaxis_title="Number of establishment",
    title_x=0.5
    )
fig.show()

### Répartition des données par note globale

In [15]:
train_df['global_rate'].value_counts()

global_rate
10.0    558106
8.0     406099
6.0     124108
9.0      92938
7.0      62841
         ...  
28.8         2
27.2         2
28.6         2
18.4         1
26.2         1
Name: count, Length: 69, dtype: int64

Valeurs manquantes

In [16]:
len(train_df[train_df['global_rate'].isna()])

169

In [17]:
fig = px.bar(train_df['global_rate'].value_counts())
fig.update_layout(
    title="Repartition of global rates",
    xaxis_title="Global rate",
    yaxis_title="Count",
    title_x=0.5
)
fig.show()

Il faudra supprimer les lignes où la note globale est supérieure à 10 car elle s'apparentent à des valeurs aberrantes.   
De même il faudra supprimer les valeurs manquantes.   
Par la suite, nous allons transformer cette feature continue en feature discrète afin de réduire la complexité de la variabilité à capturer grâce au modèle.

### Méthodologie suivie à l'issue de l'exploration des données

Suite à une rapide analyse préliminaire des données, il apparait que :   
- La répartition par établissement présente un fort déséquilibre de classes et ne pourra être utilisée comme feature dans un modèle de prédiction de la note globale.   
- La target (global_rate) présente des valeurs aberrantes et des valeurs manquantes à supprimer dans le jeu train. Par ailleurs, faisant l'hypothèse que la concordance entre le commentaire et la note ne se joue pas à un niveau aussi fin qu'un interval continu de 0 à 10, nous allons transformer cette variable en variable discrète en suivant la typologie utilisée pour le calcul du NPS (Promoter, Neutre, Détracteur).
- La feature texte (review_text) présente des valeurs manquantes à supprimer dans les jeux train et test, ainsi que des caractères spéciaux (artefacts d'acquisition) à supprimer également.  

Nous allons donc effectuer effectuer un premier nettoyage des jeux train et test comme suit :   
- Suppression des valeurs aberrantes et manquantes dans la target (global_rate).
- Suppression des valeurs manquantes de la feature texte.
- Séparation du jeu train en train et validation (on va prendre 20% du jeu total dans le jeu de validation) et vérification de la répartition des notes dans chaque jeu.

Puis, nous allons appliquer la méthodologie suivante :
1. Entraînement du modèle :
- Nettoyage de la feature texte (effectué lors du fit du modèle) avec suppression des catactères spéciaux, des chiffres et des stopwords.
- Transformation de la target en suivant la typologie du NPS (Promoteur: global_rate>=9, Neutre: 7>=global_rate<9, Détracteur: global_rate<7).
- Construction d'une pipeline avec un premier modèle NLP (ici Tfidf) et un second modèle de classification (Logistic Regression).
- Fit de la pipeline en effectuant une cross validation et affichage de l'accuracy score, de la matrice de confusion et du rapport de classification sur le jeu de validation.
- Sauvegarde du modèle
2. Prédiction :
- Chargement du modèle sauvegardé et du jeu test
- Transformation de la target et prédiction sur le jeu test
- Enregistrement des prédictions

### Suppression des valeurs manquantes

In [18]:
train_df.dropna(subset=['review_text'], inplace=True)
train_df.dropna(subset=['global_rate'], inplace=True)
test_df.dropna(subset=['review_text'], inplace=True)

### Suppression des valeurs aberrantes

In [19]:
indices_to_drop = train_df[train_df['global_rate'] > 10].index
train_df.drop(indices_to_drop, inplace=True)

### Séparation du train en train et validation

In [20]:
validation_ratio = 0.2
train_index = train_test_split(
    train_df['review_text'], random_state=42, test_size=0.2
)[0].index
val_index = train_df.index.difference(train_index)
val_df = train_df.loc[val_index]
train_df = train_df.loc[train_index]

### Vérification de la répartition de la target

In [21]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=train_df['global_rate'].value_counts().index,
    y=train_df['global_rate'].value_counts().values,
    name='Train'
))

fig.add_trace(go.Bar(
    x=val_df['global_rate'].value_counts().index,
    y=val_df['global_rate'].value_counts().values,
    name='Validation'
))

fig.update_layout(
    title="Repartition of global rates",
    xaxis_title="Global rate",
    yaxis_title="Count",
    title_x=0.5
)

fig.show()

La répartition des notes est homogène entre le jeu train et test.

### Tailles des nouveaux jeux de données

In [22]:
train_df.shape

(667240, 5)

In [23]:
val_df.shape

(166811, 5)

In [24]:
test_df.shape

(228451, 4)

### Enregistrement des jeux de données sans valeurs manquantes

In [25]:
with gzip.open('../data/train_no_nan.pkl.gz', 'wb') as f:
    pickle.dump(train_df, f)

with gzip.open('../data/val_no_nan.pkl.gz', 'wb') as f:
    pickle.dump(val_df, f)

with gzip.open('../data/test_no_nan.pkl.gz', 'wb') as f:
    pickle.dump(test_df, f)
    
train_df.to_csv('../data/train_no_nan.csv', index=False)
val_df.to_csv('../data/val_no_nan.csv', index=False)
test_df.to_csv('../data/test_no_nan.csv', index=False)